<a href="https://colab.research.google.com/github/entanglesoftware/ai-learning/blob/main/langchain/news_article_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install langchain;
!pip install chat_models
!pip install openai
!pip install deeplake
!pip install transformers
!pip install -U langchain-community
!pip install newspaper3k
!pip install lxml_html_clean

ERROR: Could not find a version that satisfies the requirement chat_models (from versions: none)
ERROR: No matching distribution found for chat_models


In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [6]:
import requests
from newspaper import Article

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.11'
}

article_url = "https://indiaai.gov.in/article/research-proves-ai-s-capacity-to-predict-future-drought-conditions"

session = requests.Session()

try:
  response = session.get(article_url, headers=headers, timeout=10)

  if response.status_code == 200:
    article = Article(article_url)
    article.download()
    article.parse()

    print(f"Title: {article.title}")
    print(f"Authors: {article.authors}")
    print(f"Publication Date: {article.publish_date}")
    print(f"Text: {article.text}")

  else:
    print(f"Failed to fetch the article. Status code: {response.status_code}")

except requests.exceptions.RequestException as e:
  print(f"Error occurred while fetching the article: {e}")

Title: Research proves AI's capacity to predict future drought conditions
Authors: []
Publication Date: None
Text: Research by the University of Sharjah, published in the journal Scientific Reports, highlights the substantial advantages of using AI models over conventional drought indices in predicting the causes and onslaught of conditions leading to a scarcity of rain and water.

According to Dr. Mohamed Abdallah, University of Sharjah's Associate Professor of Civil Engineering and the study's lead author, the results were promising. The AI models were able to predict future drought conditions with high accuracy. The AI models strongly correlated with multiple drought indicators and consistently outperformed existing indices.

The research's significance lies in its potential to enable authorities to monitor the drought and provide proper response strategies. The authors integrated AI into drought prediction, providing what they present as more reliable forecasts than hitherto availa

In [20]:
from langchain.schema import (
    HumanMessage
)

article_title = article.title
article_text = article.text

template = """You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

Title: {article_title}

{article_text}
==================================

Write a summary of the previous article."""

prompt = template.format(article_title=article_title, article_text=article_text)

# HumanMessage is a sructural data format that capture user messages within chat-based interactions.
messages = [HumanMessage(content=prompt)]

bulletTemplate = """You are an advanced AI assistant that summarizes online articles into bulleted lists.

Here's the article you want to summarize.

Title: {article_title}

{article_text}
==================================

Now, provide a summarized version of the article in a bulleted list format."""

bulletPrompt = template.format(article_title=article_title, article_text=article_text)

# HumanMessage is a sructural data format that capture user messages within chat-based interactions.
bulletMessages = [HumanMessage(content=bulletPrompt)]

frenchTemplate = """You are an advanced AI assistant that summarizes online articles into bulleted lists in French.

Here's the article you need to summarize.

==================================
Title: {article_title}

{article_text}
==================================

Now, provide a summarized version of the article in a bulleted list format, and translate it into French."""

frenchPrompt = frenchTemplate.format(article_title=article_title, article_text=article_text)

# HumanMessage is a sructural data format that capture user messages within chat-based interactions.
frenchMessages = [HumanMessage(content=frenchPrompt)]


In [9]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-4-turbo", temperature=0)

In [21]:
# generate summary in normal format
summary = chat(messages)
print(summary.content)

Title: Research proves AI's capacity to predict future drought conditions

Researchers at the University of Sharjah have published a study in the journal Scientific Reports demonstrating the effectiveness of artificial intelligence (AI) models in predicting drought conditions more accurately than traditional drought indices. Led by Dr. Mohamed Abdallah, the study shows that AI models can effectively correlate with multiple drought indicators and outperform existing methods in forecasting droughts.

The study highlights the limitations of traditional drought indices, which often fail to capture the complex nature of drought phenomena and the varying hydroclimatic conditions across different regions. By training AI models with historical climate data and soil moisture levels, the researchers developed new AI-based drought indices that provide enhanced forecasting accuracy, particularly in capturing extreme climatic changes.

This advancement in drought prediction is crucial for regions f

In [22]:
# generate summary in bulleted format
bulletSummary = chat(bulletMessages)
print(bulletSummary.content)

Title: Research proves AI's capacity to predict future drought conditions

Researchers at the University of Sharjah have published a study in the journal Scientific Reports demonstrating the effectiveness of artificial intelligence (AI) models in predicting drought conditions more accurately than traditional drought indices. Led by Dr. Mohamed Abdallah, the study shows that AI models can effectively correlate with multiple drought indicators and outperform existing methods in forecasting droughts.

The study highlights the limitations of traditional drought indices, which fail to capture the complex nature of drought phenomena and the varying hydroclimatic conditions across different regions. By utilizing advanced AI models trained on historical climate data and soil moisture levels, the researchers developed new AI-based drought indices that provide more reliable and precise forecasts.

This breakthrough has significant implications for drought management, enabling better planning and

In [18]:
# generate summary in bulleted french format
frenchBulletSummary = chat(frenchMessages)
print(frenchBulletSummary.content)

- Titre de l'article : La recherche prouve la capacité de l'IA à prédire les conditions futures de sécheresse
- Recherche menée par l'Université de Sharjah, publiée dans le journal Scientific Reports
- Avantages substantiels des modèles d'IA par rapport aux indices de sécheresse conventionnels pour prédire les causes et l'arrivée de conditions de pénurie d'eau
- Dr. Mohamed Abdallah, professeur associé en génie civil à l'Université de Sharjah, est l'auteur principal de l'étude
- Les modèles d'IA ont prédit avec une grande précision les conditions futures de sécheresse, surpassant les indices existants
- Importance de la recherche : permettre aux autorités de surveiller la sécheresse et de fournir des stratégies de réponse appropriées
- Les auteurs ont développé un nouvel indice de sécheresse météorologique basé sur divers modèles d'IA
- Limitations des indices traditionnels de sécheresse soulignées; les modèles d'IA utilisent des données climatiques historiques et des niveaux d'humidit